In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 34.2 MB/s 
     |████████████████████████████████| 7.6 MB 30.1 MB/s 
     |████████████████████████████████| 182 kB 78.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 9.5 MB/s 
     |████████████████████████████████| 132 kB 7.3 MB/s 
     |████████████████████████████████| 213 kB 56.0 MB/s 
     |████████████████████████████████| 127 kB 64.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!rm *
!wget https://raw.githubusercontent.com/chi-yan/data/main/Corona_NLP_test.csv > Corona_NLP_test.csv
!wget https://raw.githubusercontent.com/chi-yan/data/main/Corona_train_short.csv > Corona_train_short.csv

rm: cannot remove 'sample_data': Is a directory
--2023-01-02 22:15:19--  https://raw.githubusercontent.com/chi-yan/data/main/Corona_NLP_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002494 (979K) [text/plain]
Saving to: ‘Corona_NLP_test.csv.1’

Corona_NLP_test.csv 100%[===================>] 979.00K  --.-KB/s    in 0.004s  

2023-01-02 22:15:19 (213 MB/s) - ‘Corona_NLP_test.csv.1’ saved [1002494/1002494]

--2023-01-02 22:15:19--  https://raw.githubusercontent.com/chi-yan/data/main/Corona_train_short.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awai

In [3]:
!cat Corona_NLP_test* > Corona_test.csv
!cat Corona_train_short* > Corona_train.csv

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer("Attention is all you need")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

{'input_ids': [101, 1335, 5208, 2116, 1110, 1155, 1128, 1444, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:

from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'Corona_train.csv', 'test': 'Corona_test.csv'}, encoding = "ISO-8859-1")


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a22acd6173ee3a27/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment'],
        num_rows: 3798
    })
})

In [7]:




def transform_labels(label):

    label = label['Sentiment']
    num = 0
    if label == 'Positive':
        num = 0
    elif label == 'Negative':
        num = 1
    elif label == 'Neutral':
        num = 2
    elif label == 'Extremely Positive':
        num = 3
    elif label == 'Extremely Negative':
        num = 4

    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example['OriginalTweet'], padding='max_length')

dataset = dataset.map(tokenize_data, batched=True)

remove_columns = ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

from transformers import AutoModelForSequenceClassification
import torch
model_path = '/content/drive/MyDrive/weights/covid_tweet_classification.pt'

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

try:
    model.load_state_dict(torch.load('/content/drive/MyDrive/weights/hotel_review_classification.pt'))
except:
    pass


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

  0%|          | 0/3798 [00:00<?, ?ex/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:

from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=1)

train_dataset = dataset['train']
eval_dataset = dataset['test']

from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [9]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1250
  Number of trainable parameters = 108314117


Step,Training Loss
500,1.397700
1000,1.044900


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1250, training_loss=1.159694677734375, metrics={'train_runtime': 923.2148, 'train_samples_per_second': 10.832, 'train_steps_per_second': 1.354, 'total_flos': 2631181424640000.0, 'train_loss': 1.159694677734375, 'epoch': 1.0})

In [10]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

<ipython-input-10-3895db985ec2>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


***** Running Evaluation *****
  Num examples = 3798
  Batch size = 8


{'eval_loss': 0.885901689529419,
 'eval_accuracy': 0.6471827277514481,
 'eval_runtime': 120.5275,
 'eval_samples_per_second': 31.511,
 'eval_steps_per_second': 3.941}

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3798
    })
})

In [13]:
trainer.save_model("model")

Saving model checkpoint to model
Configuration saved in model/config.json
Model weights saved in model/pytorch_model.bin


In [14]:
text = ["Find out how you can protect yourself and loved ones from #coronavirus. ?", "this is another sentence and blah blah blah", "this is a very happy tweet", "corona sucks", "i love you"]
encoding = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

In [15]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("model")

loading configuration file model/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,


In [16]:
outputs = model(**encoding)
predictions = outputs.logits.argmax(-1)
predictions

tensor([3, 2, 3, 2, 3])

In [17]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.1242, -1.8559, -1.5643,  3.9902, -2.1312],
        [ 0.8781,  0.3850,  3.1926, -2.0562, -2.2475],
        [ 1.1101, -1.8775, -1.5785,  3.9281, -2.1078],
        [ 0.4164,  1.1281,  2.8207, -2.3048, -1.8836],
        [ 0.7848, -1.8299, -1.4466,  3.8802, -1.8222]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)